<a href="https://colab.research.google.com/github/LorenzoAgnolucci/BERT_for_ABSA/blob/master/BERT_for_ABSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---


# BERT-pair


---



In [1]:
#@title Choose a dataset and a task { run: "auto", display-mode: "form" }
base_dir = "C:\\Users\\prave\\Repos\\BERT_for_ABSA\\" #@param {type:"string"}
dataset_type = "sentihood" #@param ["sentihood", "semeval2014"]
task = "NLI_M" #@param ["QA_M", "NLI_M", "QA_B", "NLI_B"]

In [2]:
import pandas as pd
import random

if dataset_type == "sentihood":
    id2label = {0: "None", 1: "Positive", 2: "Negative"}
    label2id = {"None": 0, "Positive": 1, "Negative": 2}
elif dataset_type == "semeval2014":
    id2label = {0: "positive", 1: "neutral", 2: "negative", 3: "conflict", 4: "none"}
    label2id = {"positive": 0, "neutral" : 1, "negative" : 2, "conflict": 3, "none": 4}

if task.endswith("B"):
    num_classes = 2
else:
    if dataset_type == "sentihood":
        num_classes = 3
    elif dataset_type == "semeval2014":
        num_classes = 5

def get_dataset(path):
    original_sentences = []
    auxiliary_sentences = []
    labels = []
    data = pd.read_csv(path, header=0, sep="\t").values.tolist()
    for row in data:
        original_sentences.append(row[1])
        auxiliary_sentences.append(row[2])
        labels.append(row[3])
    return original_sentences, auxiliary_sentences, labels

train_original_sentences, train_auxiliary_sentences, train_labels = get_dataset(f"{base_dir}\\data\\{dataset_type}\\BERT-pair\\train_{task}.csv")
if dataset_type == "sentihood":
    val_original_sentences, val_auxiliary_sentences, val_labels = get_dataset(f"{base_dir}\\data\\{dataset_type}\\BERT-pair\\dev_{task}.csv")
elif dataset_type == "semeval2014":
    val_original_sentences, val_auxiliary_sentences, val_labels = get_dataset(f"{base_dir}\\data\\{dataset_type}\\BERT-pair\\test_{task}.csv")
test_original_sentences, test_auxiliary_sentences, test_labels = get_dataset(f"{base_dir}\\data\\{dataset_type}\\BERT-pair\\test_{task}.csv")

In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_original_sentences, train_auxiliary_sentences, truncation=True, padding=True)
val_encodings = tokenizer(val_original_sentences, val_auxiliary_sentences, truncation=True, padding=True)
test_encodings = tokenizer(test_original_sentences, test_auxiliary_sentences, truncation=True, padding=True)

C:\Users\prave\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch

class ABSA_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ABSA_Dataset(train_encodings, train_labels)
val_dataset = ABSA_Dataset(val_encodings, val_labels)
test_dataset = ABSA_Dataset(test_encodings, test_labels)

In [5]:
import sys
if base_dir not in sys.path:
    sys.path.insert(0, f'{base_dir}/')
import numpy as np
from scipy.special import softmax
import evaluation


def get_test_labels(data_dir, dataset_type):
    original_sentences = []
    auxiliary_sentences = []
    labels = []
    data = pd.read_csv(f"{data_dir}/{dataset_type}/BERT-pair/test_NLI_M.csv", header=0, sep="\t").values.tolist()
    for row in data:
        labels.append(row[3])
    return labels


def get_predictions(data, task, dataset_type):
    predicted_labels = []
    scores = []
    if task.endswith("B"):
        if dataset_type == "sentihood":
            if task.endswith("B"):
                count_aspect_rows = 0
                current_aspect_scores = []
                for row in data:
                    current_aspect_scores.append(row[2])
                    count_aspect_rows += 1
                    if count_aspect_rows % 3 == 0:
                        sum_current_aspect_scores = np.sum(current_aspect_scores)
                        current_aspect_scores = [score / sum_current_aspect_scores for score in current_aspect_scores]
                        scores.append(current_aspect_scores)
                        predicted_labels.append(np.argmax(current_aspect_scores))
                        current_aspect_scores = []
        elif dataset_type == "semeval2014":
            if task.endswith("B"):
                count_aspect_rows = 0
                current_aspect_scores = []
                for row in data:
                    current_aspect_scores.append(row[2])
                    count_aspect_rows += 1
                    if count_aspect_rows % 5 == 0:
                        sum_current_aspect_scores = np.sum(current_aspect_scores)
                        current_aspect_scores = [score / sum_current_aspect_scores for score in current_aspect_scores]
                        scores.append(current_aspect_scores)
                        predicted_labels.append(np.argmax(current_aspect_scores))
                        current_aspect_scores = []
    return predicted_labels, scores


if dataset_type == "sentihood":
    def compute_metrics(predictions):
        scores = [softmax(prediction) for prediction in predictions[0]]
        predicted_labels = [np.argmax(x) for x in scores]
        if task.endswith("B"):
            data = np.insert(scores, 0, predicted_labels, axis=1)
            predicted_labels, scores = get_predictions(data, task, dataset_type)
        test_labels = get_test_labels(f"{base_dir}/data", dataset_type)
        metrics = {}
        metrics["strict_acc"] = evaluation.compute_sentihood_aspect_strict_accuracy(test_labels, predicted_labels)
        metrics["F1"] = evaluation.compute_sentihood_aspect_macro_F1(test_labels, predicted_labels)
        metrics["aspect_AUC"] = evaluation.compute_sentihood_aspect_macro_AUC(test_labels, scores)
        sentiment_macro_AUC, sentiment_accuracy = evaluation.compute_sentihood_sentiment_classification_metrics(test_labels, scores)
        metrics["sentiment_acc"] = sentiment_accuracy
        metrics["sentiment_AUC"] = sentiment_macro_AUC
        return metrics

elif dataset_type == "semeval2014":
    def compute_metrics(predictions):
        scores = [softmax(prediction) for prediction in predictions[0]]
        predicted_labels = [np.argmax(x) for x in scores]
        if task.endswith("B"):
            data = np.insert(scores, 0, predicted_labels, axis=1)
            predicted_labels, scores = get_predictions(data, task, dataset_type)
        test_labels = get_test_labels(f"{base_dir}/data", dataset_type)
        metrics = {}
        p, r, f1 = evaluation.compute_semeval_PRF(test_labels, predicted_labels)
        metrics["P"] = p
        metrics["R"] = r
        metrics["F1"] = f1
        metrics["4-way"] = evaluation.compute_semeval_accuracy(test_labels, predicted_labels, scores, 4)
        metrics["3-way"] = evaluation.compute_semeval_accuracy(test_labels, predicted_labels, scores, 3)
        metrics["binary"] = evaluation.compute_semeval_accuracy(test_labels, predicted_labels, scores, 2)
        return metrics

In [19]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, BertConfig

from transformers import logging
logging.set_verbosity_debug()

epochs = 4
batch_size = 24
num_steps = len(train_dataset) * epochs // batch_size
warmup_steps = num_steps // 10  # 10% of the training steps
save_steps = num_steps // epochs    # Save a checkpoint at the end of each epoch


training_args = TrainingArguments(
    output_dir = f'{base_dir}models\\{dataset_type}\\\BERT-pair\\{task}\\',          
    num_train_epochs = epochs,              
    per_device_train_batch_size = batch_size,  
    per_device_eval_batch_size = batch_size,   
    warmup_steps = warmup_steps,   
    weight_decay = 0.01,               
    logging_dir = f'{base_dir}logs\\{dataset_type}\\BERT-pair\\{task}\\',            
    logging_steps = 10,
    evaluation_strategy = 'epoch',
    learning_rate = 2e-5,
    save_steps = save_steps
)

config = BertConfig.from_pretrained(
    'bert-base-uncased',
    architectures = ['BertForSequenceClassification'],
    hidden_size = 768,
    num_hidden_layers = 12,
    num_attention_heads = 12,
    hidden_dropout_prob = 0.1,
    num_labels = num_classes
)    

load_finetuned_model = False
if not load_finetuned_model:
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

    trainer = Trainer(
        model=model,                         
        args=training_args,                  
        train_dataset=train_dataset,         
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics             
    )
    trainer.train()

    model.save_pretrained(f"{base_dir}models\\{dataset_type}\\BERT-pair\\{task}\\last_step")

else:
    model = BertForSequenceClassification.from_pretrained(f"{base_dir}models\\{dataset_type}\\BERT-pair\\{task}\\last_step")

    trainer = Trainer(
        model=model,                         
        args=training_args,                  
        train_dataset=train_dataset,         
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics             
    )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at C:\Users\prave\.cache\huggingface\hub\models--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\config.json
Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_e

{'loss': 1.0175, 'grad_norm': 12.165437698364258, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.02}


  1%|          | 20/2504 [06:13<16:01:27, 23.22s/it]

{'loss': 0.9781, 'grad_norm': 14.183942794799805, 'learning_rate': 1.6000000000000001e-06, 'epoch': 0.03}


  1%|          | 30/2504 [09:55<14:49:05, 21.56s/it]

...

 17%|█▋        | 420/2504 [3:03:06<16:44:13, 28.91s/it]

{'loss': 0.3404, 'grad_norm': 6.490188121795654, 'learning_rate': 1.8491570541259985e-05, 'epoch': 0.67}


 17%|█▋        | 430/2504 [3:07:48<16:30:46, 28.66s/it]

{'loss': 0.2398, 'grad_norm': 3.8625547885894775, 'learning_rate': 1.840283939662822e-05, 'epoch': 0.69}


 17%|█▋        | 433/2504 [3:09:09<15:41:08, 27.27s/it]

In [ ]:
evaluation_result = trainer.evaluate(test_dataset)
print(evaluation_result)



---


# BERT-pair


---



In [ ]:
import pandas as pd

results = trainer.predict(test_dataset)

scores = [softmax(prediction) for prediction in results.predictions]
predicted_labels = [np.argmax(x) for x in scores]

In [ ]:
csv_output = np.insert(scores, 0, predicted_labels, axis=1)
df = pd.DataFrame(csv_output)
df[0] = df[0].astype("int")
if task.endswith("B"):
    header = ["predicted_label", "no", "yes"]
else:
    header = ["predicted_label"]
    for label in label2id.keys():
        header.append(label)
df.to_csv(f"{base_dir}/results/{dataset_type}/BERT-pair/{task}.csv", index=False, header=header)

In [ ]:
import sys
if base_dir not in sys.path:
    sys.path.insert(0, f'{base_dir}/')
import evaluation
evaluation.main(task, dataset_type, f"{base_dir}/data", f"{base_dir}/results")



---


# BERT-single


---



In [ ]:
#@title Choose a dataset and a task { run: "auto", display-mode: "form" }
base_dir = "/gdrive/MyDrive/Machine_Learning" #@param {type:"string"}
dataset_type = "semeval2014" #@param ["sentihood", "semeval2014"]
task = "single" #@param ["single"]

In [ ]:
import pandas as pd
import random


if dataset_type == "sentihood":
    id2label = {0: "None", 1: "Positive", 2: "Negative"}
    label2id = {"None": 0, "Positive": 1, "Negative": 2}
elif dataset_type == "semeval2014":
    id2label = {0: "positive", 1: "neutral", 2: "negative", 3: "conflict", 4: "none"}
    label2id = {"positive": 0, "neutral" : 1, "negative" : 2, "conflict": 3, "none": 4}

if dataset_type == "sentihood":
    num_classes = 3
    locations = ["location_1_", "location_2_"]
    aspects = ["general", "price", "safety", "transit location"]
elif dataset_type == "semeval2014":
    num_classes = 5
    locations = [""]
    aspects = ["ambience", "anecdotes", "food", "price", "service"]


def get_dataset(path):
    original_sentences = []
    labels = []
    data = pd.read_csv(path, header=0, sep="\t").values.tolist()
    for row in data:
        original_sentences.append(row[1])
        labels.append(row[3])
    return original_sentences, labels


train_original_sentences = {}
train_labels = {}
val_original_sentences = {}
val_labels = {}
test_original_sentences = {}
test_labels = {}

for location in locations:
    train_original_sentences[location] = {}
    train_labels[location] = {}
    val_original_sentences[location] = {}
    val_labels[location] = {}
    test_original_sentences[location] = {}
    test_labels[location] = {}
    for aspect in aspects:
        train_original_sentences[location][aspect], train_labels[location][aspect] = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-single/{location}{aspect}/train.csv")
        if dataset_type == "sentihood":
            val_original_sentences[location][aspect], val_labels[location][aspect] = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-single/{location}{aspect}/dev.csv")
        elif dataset_type == "semeval2014":
            val_original_sentences[location][aspect], val_labels[location][aspect] = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-single/{location}{aspect}/test.csv")
        test_original_sentences[location][aspect], test_labels[location][aspect] = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-single/{location}{aspect}/test.csv")

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = {}
val_encodings = {}
test_encodings = {}
for location in locations:
    train_encodings[location] = {}
    val_encodings[location] = {}
    test_encodings[location] = {}
    for aspect in aspects:
        train_encodings[location][aspect] = tokenizer(train_original_sentences[location][aspect], truncation=True, padding=True)
        val_encodings[location][aspect] = tokenizer(val_original_sentences[location][aspect], truncation=True, padding=True)
        test_encodings[location][aspect] = tokenizer(test_original_sentences[location][aspect], truncation=True, padding=True)

In [ ]:
import torch

class ABSA_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = {}
val_dataset = {}
test_dataset = {}
for location in locations:
    train_dataset[location] = {}
    val_dataset[location] = {}
    test_dataset[location] = {}
    for aspect in aspects:
        train_dataset[location][aspect] = ABSA_Dataset(train_encodings[location][aspect], train_labels[location][aspect])
        val_dataset[location][aspect] = ABSA_Dataset(val_encodings[location][aspect], val_labels[location][aspect])
        test_dataset[location][aspect] = ABSA_Dataset(test_encodings[location][aspect], test_labels[location][aspect])

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, BertConfig
from transformers import logging
import gc
import pandas as pd
import numpy as np
from scipy.special import softmax

logging.set_verbosity_debug()

epochs = 4
batch_size = 24

header = ["predicted_label"]
for label in label2id.keys():
    header.append(label)

config = BertConfig.from_pretrained(
        'bert-base-uncased',
        architectures = ['BertForSequenceClassification'],
        hidden_size = 768,
        num_hidden_layers = 12,
        num_attention_heads = 12,
        hidden_dropout_prob = 0.1,
        num_labels = num_classes
    )    

for location in locations:
    for aspect in aspects:
        num_steps = len(train_dataset[location][aspect]) * epochs // batch_size
        warmup_steps = num_steps // 10  # 10% of the training steps
        save_steps = num_steps // epochs    # Save a checkpoint at the end of each epoch


        training_args = TrainingArguments(
            output_dir = f'{base_dir}/models/{dataset_type}/BERT-single/{location}{aspect}/',          
            num_train_epochs = epochs,              
            per_device_train_batch_size = batch_size,  
            per_device_eval_batch_size = batch_size,   
            warmup_steps = warmup_steps,   
            weight_decay = 0.01,               
            logging_dir = f'{base_dir}/logs/{dataset_type}/BERT-single/{location}{aspect}/',            
            logging_steps = 10,
            evaluation_strategy = 'epoch',
            learning_rate = 2e-5,
            save_steps = save_steps,
            seed=21
        )

        model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

        trainer = Trainer(
            model=model,                         
            args=training_args,                  
            train_dataset=train_dataset[location][aspect],         
            eval_dataset=val_dataset[location][aspect]             
        )

        trainer.train()

        model.save_pretrained(f"{base_dir}/models/{dataset_type}/BERT-single/{location}{aspect}/last_step")

        results = trainer.predict(test_dataset[location][aspect])

        scores = [softmax(prediction) for prediction in results.predictions]
        predicted_labels = [np.argmax(x) for x in scores]

        csv_output = np.insert(scores, 0, predicted_labels, axis=1)
        df = pd.DataFrame(csv_output)
        df[0] = df[0].astype("int")
        df.to_csv(f"{base_dir}/results/{dataset_type}/BERT-single/{location}{aspect}.csv", index=False, header=header)

        del training_args
        del model
        del trainer
        del results
        del scores
        del predicted_labels
        del csv_output
        del df
        gc.collect()

In [ ]:
import sys
if base_dir not in sys.path:
    sys.path.insert(0, f'{base_dir}/')

import evaluation
evaluation.main(task, dataset_type, f"{base_dir}/data", f"{base_dir}/results")

In [ ]:
location = "location_1_"
aspect = "general"

# Accessing tokenized training data
tokenized_data = train_encodings[location][aspect]

In [ ]:
import torch

input_ids = torch.tensor(tokenized_data['input_ids'])
attention_mask = torch.tensor(tokenized_data['attention_mask'])

inputs = {
    'input_ids': input_ids.unsqueeze(0),  # Add batch dimension if not already present
    'attention_mask': attention_mask.unsqueeze(0)  # Add batch dimension if not already present
}

In [ ]:
with torch.no_grad():  # Disable gradient calculation to save memory and computations
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state